In [1]:
import sys, os

from debugger.debugger_options import DebugOptions

MARIPOSA_PATH = "/home/yizhou7/mariposa/"

sys.path.append(MARIPOSA_PATH + "src")
os.chdir(MARIPOSA_PATH)

from benchmark_consts import TIMEOUT_TARGETS
from debugger.edit_tracker import EditTracker
from debugger.tree_node import NodeRef
from debugger.debugger import DbgMode, Debugger, TimeoutDebugger
from debugger.bench_viewer import BenchViewer
from tabulate import tabulate
from tqdm import tqdm
import numpy as np
import networkx as nx


In [3]:
options = DebugOptions()
options.verbose = True
dbg = TimeoutDebugger(TIMEOUT_TARGETS[0], options)
scores = dbg.rank_edits()

[INFO] [init] resolved to data/projs/bench_unstable/base.z3/d_fvbkv--lib-Buckets-BucketWeights.i.dfy.Impl__BucketWeights.__default.WeightKeyListFlatten.smt2 
[INFO] [init] dbg root: dbg/1d57b073cc/ 
[INFO] [graph] found dbg/1d57b073cc/graphs/rename.1685587427039822516.txt 
[INFO] [stats] found dbg/1d57b073cc/stats/rename.1685587427039822516.txt 
7dc12e61fd83b6973c4cb531bc133972.pickle


[DEBUG] [edit] proof path: dbg/1d57b073cc/proofs/rename.9325809803465583249.proof 
[DEBUG] [edit] trace path: dbg/1d57b073cc/traces/rename.1685587427039822516 


[WARN] [differ] qid constructor_accessor_axiom not found in dbg/1d57b073cc/orig.smt2 
[WARN] [differ] qid <null> not found in dbg/1d57b073cc/orig.smt2 
mariposa_qid_92 {<EditAction.ERROR: 'ERROR'>}
mariposa_qid_347 {<EditAction.ERROR: 'ERROR'>}
mariposa_qid_381 {<EditAction.ERROR: 'ERROR'>}
mariposa_qid_119 {<EditAction.ERROR: 'ERROR'>}
mariposa_qid_1814 {<EditAction.ERROR: 'ERROR'>}
mariposa_qid_356 {<EditAction.ERROR: 'ERROR'>}


In [6]:
for (qname, score) in scores:
    if score == 0:
        continue
    actions = dbg.editor.get_quant_actions(qname)
    print(f"{qname}: {score}")
    print(actions)

mariposa_qid_404: 66506.477799146
{<EditAction.ERASE: 'erase'>}
mariposa_qid_4028: 45009.279008075195
{<EditAction.INST_REPLACE: 'inst_replace'>, <EditAction.INST_KEEP: 'inst_keep'>}
mariposa_qid_1286: 38382.25402255223
{<EditAction.INST_KEEP: 'inst_keep'>}
mariposa_qid_1777: 35004.66539905284
{<EditAction.INST_REPLACE: 'inst_replace'>, <EditAction.INST_KEEP: 'inst_keep'>}
mariposa_qid_1783: 34271.47592647577
{<EditAction.INST_REPLACE: 'inst_replace'>, <EditAction.INST_KEEP: 'inst_keep'>}
mariposa_qid_1779: 31914.635206571467
{<EditAction.INST_REPLACE: 'inst_replace'>, <EditAction.INST_KEEP: 'inst_keep'>}
mariposa_qid_1785: 28623.001005987964
{<EditAction.INST_REPLACE: 'inst_replace'>, <EditAction.INST_KEEP: 'inst_keep'>}
mariposa_qid_1782: 26637.267292296365
{<EditAction.ERASE: 'erase'>}
mariposa_qid_3324: 24042.520343243108
{<EditAction.INST_REPLACE: 'inst_replace'>, <EditAction.INST_KEEP: 'inst_keep'>}
mariposa_qid_50: 24002.910578193572
{<EditAction.INST_REPLACE: 'inst_replace'>, <